<a href="https://colab.research.google.com/github/root-epifit/madmo-adv/blob/my_exercise/homeworks_my_solutions/hw2_poetry_generation_my_solutiuon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2

## Almost Shakespeare

Let's try to generate some Shakespeare poetry using RNNs. The sonnets file is available in the notebook directory.

Text generation can be designed in several steps:
    
1. Data loading
2. Dictionary generation
3. Data preprocessing
4. Model (neural network) training
5. Text generation (model evaluation)

### Data loading

Shakespeare sonnets are awailable at this [link](http://www.gutenberg.org/ebooks/1041?msg=welcome_stranger). In addition, they are stored in the same directory as this notebook (`sonnetes.txt`).

Simple preprocessing is already done for you in the next cell: all technical info is dropped.

**Alternatively**

You could use file `onegin.txt` with Russian texts or your natve language poetry to be able to assess results quality.

**Note: In case of Onegin text you need to adjust reading procedure yourself!!!** (this file has a bit different format than `sonnets.txt`)

In [1]:
!wget -nc https://raw.githubusercontent.com/v-goncharenko/madmo-adv/55d929befa12370fc18109f5333f7cf000ea27ce/homeworks/sonnets.txt
!wget -nc https://raw.githubusercontent.com/v-goncharenko/madmo-adv/55d929befa12370fc18109f5333f7cf000ea27ce/homeworks/onegin.txt

--2021-11-17 13:54:13--  https://raw.githubusercontent.com/v-goncharenko/madmo-adv/55d929befa12370fc18109f5333f7cf000ea27ce/homeworks/sonnets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119748 (117K) [text/plain]
Saving to: ‘sonnets.txt’

sonnets.txt         100%[===================>] 116.94K  --.-KB/s    in 0.02s   

2021-11-17 13:54:13 (5.41 MB/s) - ‘sonnets.txt’ saved [119748/119748]

--2021-11-17 13:54:13--  https://raw.githubusercontent.com/v-goncharenko/madmo-adv/55d929befa12370fc18109f5333f7cf000ea27ce/homeworks/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.

In [2]:
with open("sonnets.txt", "r") as iofile:
    text = iofile.readlines()

TEXT_START = 45
TEXT_END = -368
text = text[TEXT_START:TEXT_END]
assert len(text) == 2616

In [3]:
text[:20]

['  From fairest creatures we desire increase,\n',
 "  That thereby beauty's rose might never die,\n",
 '  But as the riper should by time decease,\n',
 '  His tender heir might bear his memory:\n',
 '  But thou, contracted to thine own bright eyes,\n',
 "  Feed'st thy light's flame with self-substantial fuel,\n",
 '  Making a famine where abundance lies,\n',
 '  Thy self thy foe, to thy sweet self too cruel:\n',
 "  Thou that art now the world's fresh ornament,\n",
 '  And only herald to the gaudy spring,\n',
 '  Within thine own bud buriest thy content,\n',
 "  And tender churl mak'st waste in niggarding:\n",
 '    Pity the world, or else this glutton be,\n',
 "    To eat the world's due, by the grave and thee.\n",
 '\n',
 '  II\n',
 '\n',
 '  When forty winters shall besiege thy brow,\n',
 "  And dig deep trenches in thy beauty's field,\n",
 "  Thy youth's proud livery so gazed on now,\n"]

In [4]:
with open("onegin.txt", "r") as iofile:
    otext = iofile.readlines()

for i, token in enumerate(otext):
    otext[i] = token.replace('\t',' ')
#TEXT_START = 45
#TEXT_END = -368
#text = text[TEXT_START:TEXT_END]
assert len(otext) == 7088

In [5]:
len(otext)

7088

In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [6]:
import string

In [7]:
# Join all the strings into one and lowercase it
# Put result into variable text.

# Your great code here

text = ''.join(text).lower()

assert len(text) == 100225, "Are you sure you have concatenated all the strings?"
assert not any([x in set(text) for x in string.ascii_uppercase]), "Uppercase letters are present"
print("OK!")

OK!


Put all the characters, that you've seen in the text, into variable `tokens`.

In [8]:
tokens = sorted(set(text))

Create dictionary `token_to_idx = {<char>: <index>}` and dictionary `idx_to_token = {<index>: <char>}`

In [14]:
# dict <index>:<char>
# Your great code here
token_to_idx = {char:i for i, char in enumerate(tokens) }

# dict <char>:<index>
# Your great code here

idx_to_token = {i:char for i, char in enumerate(tokens)}

*Comment: in this task we have only 38 different tokens, so let's use one-hot encoding.*

### Building the model

Now we want to build and train recurrent neural net which would be able to something similar to Shakespeare's poetry.

Let's use vanilla RNN, similar to the one created during the lesson.

### Data preparation (text normalization)

In [13]:
import string

In [2]:
with open("onegin.txt", "r") as iofile:
    text = iofile.readlines()

In [14]:
# Convert to lowercase & replace '\t', \n' with ''
text = [token.lower().
        replace('\t', '').
        replace(',\n',' ').
        replace('.\n',' ').
        replace(':\n',' ').
        replace('\n',' ').
        replace('«',' ').
        replace('»',' ').
        replace(', ',' ').
        replace('. ',' ').
        replace('! ',' ').
        replace('? ',' ').
        replace(': ',' ')  for token in text]

# Drop empy strings
text = [token for token in text if token != ' ']
    
# Convert to string
text = ''.join(text)

text

'i  мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог его пример другим наука; но боже мой какая скука с больным сидеть и день и ночь не отходя ни шагу прочь какое низкое коварство полуживого забавлять ему подушки поправлять печально подносить лекарство вздыхать и думать про себя когда же черт возьмет тебя  ii так думал молодой повеса летя в пыли на почтовых всевышней волею зевеса наследник всех своих родных — друзья людмилы и руслана с героем моего романа без предисловий сей же час позвольте познакомить вас онегин добрый мой приятель родился на брегах невы где может быть родились вы или блистали мой читатель; там некогда гулял и я но вреден север для меня iii служив отлично-благородно долгами жил его отец давал три бала ежегодно и промотался наконец судьба евгения хранила сперва madame за ним ходила потом monsieur ее сменил; ребенок был резов но мил monsieur l’abbe€ француз убогой чтоб не измучилось дитя учил его всему шутя не доку

In [ ]:
!pip install pymorphy2

In [23]:
import pandas as pd
from string import punctuation #сборник символов пунктуации
from nltk.tokenize import word_tokenize #для токенизации по словам
from nltk.corpus import stopwords #сборник стоп-слов
import pymorphy2 #для морфологическтого анализа текста
from nltk.probability import FreqDist #используется для кодирования «частотных распределений»

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [45]:
# Делим текст на токены, выкидываем из них знаки препинания 

tokens = word_tokenize(text)

#stopword = stopwords.words('russian') + [a for a in punctuation]
stopword = [a for a in punctuation]
tokens = [word for word in tokens if word not in stopword]

# Преобразовываем слова в словоформы
morph = pymorphy2.MorphAnalyzer()
tokens = [morph.parse(item)[0].normal_form for item in tokens]

print(f'Num of tokens = {len(tokens)}')
print(tokens[:20])

Num of tokens = 22996
['i', 'мой', 'дядя', 'самый', 'честной', 'правило', 'когда', 'не', 'в', 'шутка', 'занемочь', 'он', 'уважать', 'себя', 'заставить', 'и', 'хороший', 'выдумать', 'не', 'мочь']


In [57]:
# token_to_idx / idx_to_token encoding
token_to_idx = {token:i for i, token in enumerate(set(tokens))}

idx_to_token = {i:token for i, token in enumerate(set(tokens))}

In [56]:
for k, v in token_to_idx.items():
    print(k,v)

Выходные данные были обрезаны до нескольких последних строк (5000).
пёстрый 167
вспоминать 168
угодный 169
говорливый 170
зала 171
исключение 172
цена 173
ступень 174
полтина 175
подслушать 176
восемь 177
guillot 178
читать 179
недоброхотство 180
глупцов… 181
струиться 182
серебриться 183
ещё 184
трепетание 185
этот 186
хлыстик 187
защита 188
охладеть 189
колыбельный 190
трещать 191
обращать 192
быстро 193
нереида 194
уж 195
стремнина 196
гладкий 197
осуждение 198
ленивец 199
трагедия 200
сделаться 201
лепет 202
почти 203
угомонить 204
открытый 205
носить 206
людская 207
отпираться 208
сырой 209
но 210
мрачный 211
смиренно 212
верста 213
оплошный 214
воспомнить 215
вкруг 216
вино 217
оживить 218
приникнуть 219
надолго 220
пролас 221
томить 222
назвать 223
звучно 224
молчание 225
раб 226
улыбка 227
неотразимый 228
учтивый 229
спроситься 230
седой 231
машинально 232
дельный 233
горишь… 234
слушать 235
прохлада 236
возражение 237
голубка 238
неправильный 239
славно 240
’ 241
посметь 242
о

In [55]:
token_to_idx.__dir__()

['__repr__',
 '__hash__',
 '__getattribute__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__iter__',
 '__init__',
 '__len__',
 '__getitem__',
 '__setitem__',
 '__delitem__',
 '__contains__',
 '__new__',
 '__sizeof__',
 'get',
 'setdefault',
 'pop',
 'popitem',
 'keys',
 'items',
 'values',
 'update',
 'fromkeys',
 'clear',
 'copy',
 '__doc__',
 '__str__',
 '__setattr__',
 '__delattr__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__dir__',
 '__class__']

In [38]:
len([item for item in uniq_tokens])

8517

In [41]:

#for ii in range(len(tokens)):
#    token_2[ii] = morph.parse(token_2[ii])[0].normal_form

len([_ for _ in set([morph.parse(item)[0].normal_form for item in tokens])])

5167

In [ ]:
# Your code here

In [36]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn


# Convert text to list of chars
def text_to_tokens(text:str) -> list:
    return [_ for _ in text]

# Convert text to sequence of idxs
def tokens_to_seq(tokens:list) -> np.array:    
    return np.array([token_to_idx[_] for _ in tokens])

# Convert sequence of idxs to text
def seq_to_text(sequence:np.array) -> str:
    return "".join([idx_to_token[_] for _ in sequence])


SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(tokens):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [38]:
#seq_to_text(tokens_to_seq(text_to_tokens("abba")))
tokens_to_seq(text_to_tokens("abba"))

array([12, 13, 13, 12])

100225

In [17]:
[char for char in text]

[' ',
 ' ',
 'f',
 'r',
 'o',
 'm',
 ' ',
 'f',
 'a',
 'i',
 'r',
 'e',
 's',
 't',
 ' ',
 'c',
 'r',
 'e',
 'a',
 't',
 'u',
 'r',
 'e',
 's',
 ' ',
 'w',
 'e',
 ' ',
 'd',
 'e',
 's',
 'i',
 'r',
 'e',
 ' ',
 'i',
 'n',
 'c',
 'r',
 'e',
 'a',
 's',
 'e',
 ',',
 '\n',
 ' ',
 ' ',
 't',
 'h',
 'a',
 't',
 ' ',
 't',
 'h',
 'e',
 'r',
 'e',
 'b',
 'y',
 ' ',
 'b',
 'e',
 'a',
 'u',
 't',
 'y',
 "'",
 's',
 ' ',
 'r',
 'o',
 's',
 'e',
 ' ',
 'm',
 'i',
 'g',
 'h',
 't',
 ' ',
 'n',
 'e',
 'v',
 'e',
 'r',
 ' ',
 'd',
 'i',
 'e',
 ',',
 '\n',
 ' ',
 ' ',
 'b',
 'u',
 't',
 ' ',
 'a',
 's',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'r',
 'i',
 'p',
 'e',
 'r',
 ' ',
 's',
 'h',
 'o',
 'u',
 'l',
 'd',
 ' ',
 'b',
 'y',
 ' ',
 't',
 'i',
 'm',
 'e',
 ' ',
 'd',
 'e',
 'c',
 'e',
 'a',
 's',
 'e',
 ',',
 '\n',
 ' ',
 ' ',
 'h',
 'i',
 's',
 ' ',
 't',
 'e',
 'n',
 'd',
 'e',
 'r',
 ' ',
 'h',
 'e',
 'i',
 'r',
 ' ',
 'm',
 'i',
 'g',
 'h',
 't',
 ' ',
 'b',
 'e',
 'a',
 'r',
 ' ',
 'h',
 'i',
 's',
 

{'\n',
 ' ',
 '!',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

Plot the loss function (axis X: number of epochs, axis Y: loss function).

In [ ]:
# Your plot code here

In [ ]:
# An example of generated text. There is no function `generate_text` in the code above.
# print(generate_text(length=500, temperature=0.2))

hide my will in thine?
  shall will in of the simend that in my sime the seave the seave the sorll the soren the sange the seall seares and and the fart the wirl the seall the songh whing that thou hall will thoun the soond beare the with that sare the simest me the fart the wirl the songre the with thy seart so for shat so for do the dost the sing the sing the sing the soond canding the sack and the farling the wirl of sore sich and that with the seare the seall so fort the with the past the wirl the simen the wirl the sores the sare


### More poetic model

Let's use LSTM instead of vanilla RNN and compare the results.

Plot the loss function of the number of epochs. Does the final loss become better?

In [ ]:
# Your beautiful code here

Generate text using the trained net with different `temperature` parameter: `(0.1, 0.2, 0.5, 1.0, 2.0)`.

Evaluate the results visually, try to interpret them.

In [ ]:
# Text generation with different temperature values here

### Saving and loading models

Save the model to the disk, then load it and generate text.
Follow guides from [this tutorial](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

You need to use `Save/Load state_dict (Recommended)` section aka save state dict.

In [ ]:
# Saving and loading code here

## Additional materials on topic

1. [Andrew Karpathy blog post about RNN.](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)\
There are several examples of genration: Shakespeare texts, Latex formulas, Linux Sourse Code and children names.
2. <a href='https://github.com/karpathy/char-rnn'> Repo with char-rnn code </a>
3. Cool repo with [PyTorch examples](https://github.com/spro/practical-pytorch`)